In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pprint
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d,MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# torchvision for pre-trained models
from torchvision import models

import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import transforms


In [2]:
master_csv = pd.read_csv(r"C:\Users\sonam\University of Canberra\4th Semester\Capstone\Models\MASTER_CSV_4.csv")
master_csv.head()
master_csv.shape


(12660, 4)

In [5]:
cwd = os.getcwd() 
folder_path = os.path.join(cwd,"session_1")


eeg_data = []
for filename in tqdm(master_csv['filename']):
    file_path = os.path.join(folder_path, filename)
    file = pd.read_csv(file_path)
    
    
    eeg_values = file.astype('float32')
    # Calculate mean and std
    mean, std = eeg_values.mean(), eeg_values.std()
    
    # Normalize EEG data
    normalized_data = (eeg_values - mean) / std
    
    eeg_data.append(normalized_data)

# Convert the list of normalized arrays to a NumPy array
eeg_numpy = np.array(eeg_data)
print(eeg_numpy.shape)



100%|████████████████████████████████████████████████████████████████████████████| 12660/12660 [05:18<00:00, 39.70it/s]


(12660, 1001, 62)


In [134]:
#eeg_data

In [135]:
eeg_numpy

array([[[ 0.8213702 ,  0.80190176,  0.74304444, ...,  0.20115194,
          0.15247741,  0.12589903],
        [-0.56378865,  0.5369889 ,  0.18104266, ..., -0.02155637,
         -0.23300526, -0.47552246],
        [-1.1365596 ,  0.50063795, -0.01067706, ..., -0.19604553,
         -0.5124569 , -0.80384374],
        ...,
        [ 0.89771086,  0.8076676 ,  0.97631395, ...,  0.7032685 ,
          0.7107719 ,  0.7856219 ],
        [ 0.74954593,  0.73610884,  0.90157044, ...,  0.5720468 ,
          0.5625846 ,  0.55087525],
        [ 0.63154274,  0.5298732 ,  0.4967377 , ...,  0.2804205 ,
          0.24401589,  0.23140208]],

       [[ 0.34617177,  0.33787128,  0.3630459 , ...,  0.3045646 ,
          0.26065728,  0.2834916 ],
        [ 0.11114397, -0.0477681 , -0.07044666, ..., -0.3479429 ,
         -0.36172593, -0.02637697],
        [ 0.02818565, -0.18522081, -0.22277236, ..., -0.86739564,
         -0.7625559 , -0.46746096],
        ...,
        [-0.01469686, -0.33014822, -0.07334253, ...,  

In [136]:
eeg_numpy.shape

(360, 400, 62)

In [137]:
# defining the target
label_emotion = master_csv['emotion'].values
print(label_emotion[:48])  # Display the first 10 elements

[1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1
 1 2 3 2 2 3 3 0 3 0 3]


In [138]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(eeg_numpy, label_emotion, test_size = 0.25, random_state = 13, stratify=label_emotion)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((270, 400, 62), (270,)), ((90, 400, 62), (90,)))

In [139]:
# converting training images into torch format
train_x = train_x.reshape(270, 1, 400, 62)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y).long()

# shape of training data
train_x.shape, train_y.shape

# converting validation images into torch format
val_x = val_x.reshape(90, 1, 400, 62)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y).long()

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([90, 1, 400, 62]), torch.Size([90]))

In [143]:
class ConvNetGray(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Linear(512 * 12 * 1, 256),  # Adjusted to match the output from convolutional layers
            nn.Linear(256, 128),
            nn.Linear(128, 64),
            nn.Linear(64, 4)
        )

    def forward(self, x):
        out = self.conv2(x)
        out = out.view(out.size(0), -1)  # Flatten the output
        out = self.fc(out)
        return out



In [144]:
# defining the model
model = ConvNetGray()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
#if torch.cuda.is_available():
device = torch.device("cpu")
model = model.to(device)
criterion = criterion.to(device)

print(model)

ConvNetGray(
  (conv2): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (7): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=

In [145]:
# batch size of the model
batch_size = 128

# number of epochs to train the model
n_epochs = 15

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
#     permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        #if torch.cuda.is_available():
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:19<00:00, 26.49s/it]


epoch: 	 1 	 training loss: 	 1.420012354850769


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:03<00:00, 21.32s/it]


epoch: 	 2 	 training loss: 	 1.0555397470792134


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:02<00:00, 20.68s/it]


...

epoch: 	 13 	 training loss: 	 0.022095393564086407


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:02<00:00, 20.77s/it]


epoch: 	 14 	 training loss: 	 0.020061676438975457


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:03<00:00, 21.11s/it]

epoch: 	 15 	 training loss: 	 0.005418449213417868


In [149]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.17s/it]

training accuracy: 	 1.0


In [150]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.73s/it]

validation accuracy: 	 0.34444444444444444
